In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0 
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import RMSprop
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Loading Data Set
Three sets are created: training, validation, and test. 
- Labels are generated based on the folder structure. Class name must correspond to the subfolder name.
- Loading in batches, of size 32, to reduce memory usage.
- Label mode is set to categorical, which means that the labels are encoded as a categorical vector.

Bilinear interploation is set to default. This specify the method used in the resizing procedure. By default aspect ratio is not perserved, i.e., the ratio between image width and height.

One hot encoding is utilized when label mode is set to categorical.


The image load documentation is available [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) and an example is available [here](https://keras.io/api/data_loading/image/).

In [2]:
train_ds = image_dataset_from_directory(
    directory="datasets/dataset_v7/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
val_ds = image_dataset_from_directory(
    directory="datasets/dataset_v7/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )

Found 430968 files belonging to 12 classes.


2023-04-10 20:14:38.395621: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30994 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:b7:00.0, compute capability: 7.0


Found 37219 files belonging to 12 classes.


# EfficientNetB0

In [3]:
# initialize wandb
run = wandb.init(project="EfficientNetB0", config={"learning_rate": 0.0256,
                                               "epochs": 100, 
                                               "momentum": 0.9,
                                               "weight_decay": 1e-5,
                                               "batch_size": 256,
                                               "input_shape": (224, 224, 3),
                                               "optimizer": "RMSProp",
                                               "loss": "categorical_crossentropy",
                                               "metrics": ["accuracy"],
                                               "verbose": 1,
                                               "name": "EfficientNetB0",
                                               "architecture": "EfficientNetB0"
                                              })

# configs
cfg = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabri-torland (nubs). Use `wandb login --relogin` to force relogin


## Build the model
The hyperparameters and fully connected layers closely resemble those employed in the 2015 paper that first presented residual networks. The paper is available here: http://arxiv.org/abs/1512.03385

In [4]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=cfg.input_shape)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
predictions = layers.Dense(len(train_ds.class_names), activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

## Compile the model

In [5]:
model.compile(optimizer=RMSprop(learning_rate=cfg.learning_rate, rho=0.9, momentum=cfg.momentum, decay=cfg.weight_decay),
              loss=cfg.loss,
              metrics=cfg.metrics)

## Train the model

In [6]:
# list of callbacks
callbacks = [
            WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            EarlyStopping(monitor='val_loss', patience=10, mode="min"),
          ]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gabrielt/.local/lib/python3.8/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/gabrielt/.local/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/disk.py", line 76, in start
    self.metrics_monitor.start()
  File "/home/gabrielt/.local/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 

In [ ]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=cfg.batch_size, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks)

Epoch 1/100


2023-04-10 20:14:55.434102: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


 1747/13468 [==>...........................] - ETA: 35:42 - loss: 2.2458 - accuracy: 0.4088

In [ ]:
wandb.finish()